In [3]:
import os
import sys

In [4]:
import string
import itertools
import math
import time
import datetime
import json
import pyodbc
import numpy as np
import pandas as pd

from functools import singledispatch
from collections import Counter

# singledispatchmethod in functools from python3.8+
try:
    from functools import singledispatchmethod
except ImportError:
    from singledispatchmethod import singledispatchmethod

# Gensim
import gensim
import gensim.corpora as corpora

from gensim.utils import simple_preprocess
from gensim.models import LdaModel, CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import nltk
#nltk.download('stopwords') #only on first install

from nltk.corpus import stopwords

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_least_angle.py:34: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_least_angle.py:164: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not 

In [5]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use','please','au', 'hi','hello','help',
                  'find','cant','one','still','ask','st','yes','im','dont','none','cannot',
                  'want','know','see','whats','could','back','would_like','wondering','able',
                  'hey','need','get','take','like','copy','nan','writing','ok','appreciate',
                  'okay','thanks','make','trying','another','havent','would','thank','yet','receive',
                  'looking','someone', 'contact_us', 'popular_questions','already','idont',
                  'ive','wanted','today','first','information','talk','person','long','work',
                  'week','regarding','questions','two','pm','sent','got','says','whether','youll',
                  'said', 'hi', 'dear', 'hello'])
stop_words.extend(['question', 'related', 'detail','found','otherwise','sorry','understand','try','rephrase','try','go','either',
                  'however','includes','wish','also','sure','may','must','including','would'])
stop_words.extend(['regard', 'give', 'let'])

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

/opt/anaconda3/lib/python3.8/site-packages/spacy/language.py:1895: UserWarning: [W123] Argument disable with value ['parser', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


In [6]:
df = pd.read_csv('Archibus WR Data_2021-2022_20221018.csv')
df

,WR_ID,PROB_TYPE,REQUESTOR,STATUS,DATE_REQUESTED,DESCRIPTION,SITE_ID,BL_ID,FL_ID,RM_ID
0,6219244,ELECTRICAL SERVICES,TAWANDA LAWYER CHITSIKE,Clo,4/8/2021 0:00,"Dear COS, the lights in the male toilet on lev...",1.0,F03,03,T302
1,6219892,PLUMBING SERVICES,SALLY CHIK,Rej,4/13/2021 0:00,The zip tap in the library staff room is not w...,2.0,G02,01,116
2,6219909,LIGHTING,ANDREW LONG BAO HUA DUONG,Clo,4/13/2021 0:00,"Lights are currently out, previously occurred ...",2.0,H04,01,124
3,6219912,BLDG ACCESSCARDS,PAUL CHRISTOPHER BROOKS,Com,4/13/2021 0:00,Can i please get access for Celine\tBoehm\t116...,1.0,A28,NaN,NaN
4,6219913,FIRE SERVICES,JOSH MCALPINE,Clo,4/13/2021 0:00,NON CRITICAL DEFECT\nThe syphonics pipe collar...,1.0,B23,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
118250,6767491,ELECTRICAL SERVICES,MARK JOSEPH BUSETTO,Com,10/7/2022 0:00,2 electrical techs with site knowledge to be ...,2.0,G12,XS,NaN
118251,6752345,PLUMBING SERVICES,ROBERT LIPSCOMBE,Com,9/28/2022 0:00,All of the filters at the main tank and the co...,13.0,E20G,01,100
118252,6752571,PLUMBING SERVICES,JOSIE CLARKE,Com,9/28/2022 0:00,Hot water not working in Mayfarm cottage 2\n\n...,11.0,S2,NaN,NaN
118253,6767679,ROOF AND GUTTER,COLLEEN RITCHARD,Com,10/7/2022 0:00,Please check leaking gutter,13.0,C08DA,RF,NaN


for category in doc:
    df.groupby['PROB_TYPE']
    return 

In [7]:
#Data cleaning
class Preprocessor:
    
    def __init__(self):
        self.punct = list(string.punctuation.replace("'", '’'))
        self.token_list = [
          '|', '------','____','If you have received this email in error ','Any use, distribution, disclosure, or copying of this email','cheers', 'regards', 'best regards', 'kind regards',
          'sent from:', 'from:', 'sent:', 'to:', 'cc:', 'subject:', 'Sender:','date: ',' outlook for', '<https://protect','This short survey will take','how satisfied or dissatisfied are you with our',
          'CISCO', '[image]', '.pdf', '[cid:', 'Giving today. Changing tomorrow.','If you have a different query, you will need to submit a new enquiry', 
          'See the impact<', 'This email plus any attachments to it are confidential. Any unauthori','Read through your handbook<http','find out what you will need to enrol in your new course<https',
          'We acknowledge and respect the', 'Please think of our environment','If you are not satisfied with this response, you can re-open the enquiry',
          'This e-mail may contain information which is confidential', 'Before commencing your research you will need to enrol, to review the details of your approved',
          'This email (including any attachments) may contain confidential','We acknowledge the traditional custodians of the land','This communication may contain information that is proprietary',
          'Views expressed in this email are those of the individual sender, and are not necessarily the views of', 'You can find out how to suspend your studies<https',
          'This email plus any attachments to it are confidential','for any other general enquiries you are welcome to contact the Student Centre',
          '=========', 'Please quote your full name and student number in all correspondence with the University.','Your student card is valid for the duration of your enrolment',
          'Please use your student email account if you are a current student for all communication with the University.','Thinking about taking a break from your studies',
          'The majority of correspondence issued by the University of Sydney is sent via email. In order to ensure','You can find more information about enrolment<',
          'To update this case please click on the link in the email below, or alternatively reply above this lin','After you have successfully enrolled, we will send a confirmation summary to your University student',
          'If you receive this email in error', 'It is strictly forbidden to share any part of this message','If you have any questions, call us on 1800 SYD UNI'
        ]
        self.token_list = [t.lower() for t in self.token_list]
    
    def __call__(self, *args, **kwargs):
        if isinstance(args[0], list):
            [self(arg) for arg in args[0]]
        elif isinstance(args[0], pd.DataFrame):
            self.replace_DESCRIPTION(args[0])
            self.remove_blacklisted_sentences(args[0])
        else:
            raise ValueError
            
    def remove_punctuations(self, text):
        for punctuation in self.punct:
            text = text.replace(punctuation, ' ')
        return text

    def str_blacklist(self, string):
        str_list = string.split('\n')
        return '\n'.join([s for s in str_list if not any(t in s.lower() for t in self.token_list) and len(s) != 0])
      
    def remove_blacklisted_sentences(self, df):
        df['DESCRIPTION'] = df['DESCRIPTION'].apply(self.str_blacklist)

    def replace_DESCRIPTION(self, df):

        df['DESCRIPTION'] = df['DESCRIPTION'].astype(str)
        
        df['DESCRIPTION'] = df['DESCRIPTION'].apply(lambda x: x.lower())
        df['DESCRIPTION'] = df['DESCRIPTION'].apply(lambda x: x.replace('\n', ' '))
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace('http\S+|www.\S+', '', regex=True)
        df['DESCRIPTION'] = df['DESCRIPTION'].apply(lambda x: x.replace('&gt;', ''))
        df['DESCRIPTION'] = df['DESCRIPTION'].apply(self.remove_punctuations)
        df['DESCRIPTION'] = df['DESCRIPTION'].apply(lambda x: str(x).replace(" s ", " "))

        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace("\r", " ", case=False)

        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(' university of sydney',' usyd ', case=False)
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(' university ', ' usyd ', case=False)
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(' uni ',' usyd ',case=False)  #do not use this line it will mess up unit as usydt
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace("sydney student","sydney_student",case=False)
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace("contact us", "contact_us", case=False)
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace("popular questions","popular_questions", case=False)
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(' center ',' centre ',case=False)
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace("student centre", "student_centre", case=False)
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace("unit of study","unit_of_study", case=False)

        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" units of study "," unit_of_study ",case=False)

        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" masters "," master ", case=False)

        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace("online enquiry", "online_enquiry", case=False)
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace("online inquiry", "online_inquiry", case=False)

        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" re enrol "," re_enrol ", case=False)
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" enroll ", " enrol ", case=False)
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" enrolment "," enrol ",case=False)
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" enroling "," enrol ",case=False)
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" enroled "," enrol ",case=False)
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" enrolled "," enrol ", case=False)

        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" re enrollment ", " re_enrol ", case=False)
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" student id " , " student_id ", case=False)
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" log into "," login ",case=False)
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" log in "," login ",case=False)

        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace("error message ", " error_message ", case=False)

        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" international students ", " international_students ", case=False)
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" international student ", " international_students ", case=False)
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" first year ", " first_year ", case=False)
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" final year ", " final_year ", case=False)

        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" census date ", " census_date ", case=False)
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" none of the above ", " none_of_the_above ", case=False)
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(' units ',' unit ', case=False)

        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" SID ", " student_id ", case=False)

        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" summer school "," summer_school ",case=False)

        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" collecting "," collection ",case=False)
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" collect "," collection ",case=False)

        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" studies "," study ",case=False)
        #new
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" students ", " student ", case=False)
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" covid", " coronavirus ", case=False)
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" corona virus ", " coronavirus ", case=False)
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" suspension ", " suspend ", case=False)
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" universitys ", " usyd ", case=False)
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" begin ", " start ", case=False)
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" face to face ", " face_to_face", case=False)
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" reenrol", " re_enrol", case=False)
        df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(" australian ", " australia ", case=False)

In [8]:
preprocessor = Preprocessor()
preprocessor(df)

In [9]:
class Corpus:
    
    def __init__(self, nlp, stop_words):
        self.stop_words = set(stop_words)
        self.nlp = nlp
        
    def __call__(self, df, key='DESCRIPTION', **kwargs):
        self.data_lemmatized(df, key)
        
    def data_words(self, series):
        return list(self.sent_to_words(series.values.tolist()))
    
    def sent_to_words(self, sentences):
        for sentence in sentences:
            yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
    
    def make_bigrams(self, bigram_mod, texts):
        return [bigram_mod[doc] for doc in texts]
    
    def remove_stopwords(self, texts):
        return [[word for word in simple_preprocess(str(doc)) if word not in self.stop_words] for doc in texts]
    
    def lemmatization(self, texts, allowed_postags=('NOUN', 'VERB')):
        """https://spacy.io/api/annotation"""
        texts_out = []
        for sent in texts:
            doc = self.nlp(" ".join(sent))
            texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        return texts_out
    
    @singledispatchmethod
    def data_lemmatized(self, series, key='DESCRIPTION', **kwargs):
        data_words = self.data_words(series)
        data_words_nostops = self.remove_stopwords(data_words)
        
        bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
        bigram_mod = gensim.models.phrases.Phraser(bigram)
        
        data_words_bigrams = self.make_bigrams(bigram_mod, data_words_nostops)
        
        return self.lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'VERB'])
    
    @data_lemmatized.register(list)
    def _(self, df_list, key='DESCRIPTION', **kwargs):
        return [self.data_lemmatized(df, key, **kwargs) for df in df_list]
    
    @data_lemmatized.register(pd.DataFrame)
    def _(self, df, key='DESCRIPTION', out_key='data_words', **kwargs):
        df[out_key] = self.data_lemmatized(df[key], key='DESCRIPTION', **kwargs)

In [10]:
corpus_constructor = Corpus(nlp, stop_words)
corpus_constructor(df)

text = df['DESCRIPTION']
text

0         dear cos  the lights in the male toilet on lev...
1         the zip tap in the library staff room is not w...
2         lights are currently out  previously occurred ...
3         can i please get access for celine\tboehm\t116...
4         non critical defect the syphonics pipe collar ...
                                ...                        
118250    2 electrical techs with site knowledge  to be ...
118251    all of the filters at the main tank and the co...
118252    hot water not working in mayfarm cottage 2  ha...
118253                          please check leaking gutter
118254    could you please provide access to the student...
Name: DESCRIPTION, Length: 118255, dtype: object

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(text)

feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()

results = pd.DataFrame(denselist, columns=feature_names)

In [ ]:
@pd.api.extensions.register_dataframe_accessor('_topic')
class TopicFrame:
    
    def __init__(self, pandas_obj):
        self._obj = pandas_obj
    
    @property
    def hierarchy(self):
        return ['u_category', 'u_wrap_up_code', 'contact_type', 'date_range']
    
    def keys_from_column(self, column):
        return self._obj[column].unique()
    
    def keys_from_columns(self, columns):
        return itertools.product(*[self.keys_from_column(col) for col in columns])
    
    def dataframe_from_keys(self, columns, keys):
        return self._obj.query(' and '.join([f'{col} == "{key}"' for col, key in zip(columns, keys)]))
    
    def iterate(self, columns=None, return_keys=False):
        columns = self.hierarchy if columns is None else columns
        for keys in self.keys_from_columns(columns):
            df = self.dataframe_from_keys(columns, keys)
            if df.shape[0] != 0:
                if return_keys:
                    yield df, {col: key for col, key in zip(columns, keys)}
                else:
                    yield df

In [ ]:
def get_corpus(dictionary, texts):
    return [dictionary.doc2bow(text) for text in texts]


def get_dictionary(texts):
    return gensim.corpora.Dictionary(texts) 


def get_optimised_lda_model(dictionary, corpus, texts, limit=14):
    try:
        top_model = LdaModel(corpus=corpus, num_topics=1, id2word=dictionary)
    except ValueError:
        dictionary = get_dictionary([['Empty']])
        top_model = LdaModel(corpus=corpus, num_topics=1, id2word=dictionary)

    top_coherence = 0

    limit = min(limit, np.unique(texts).shape[0])
    for num_topics in range(2, limit):
        model = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        coherence = CoherenceModel(model=top_model, texts=texts, dictionary=dictionary, coherence='c_v').get_coherence()

        if np.isnan(coherence):
            return top_model
        elif coherence > top_coherence:
            top_model, top_coherence = model, coherence

    return top_model


class TopicConstructor:
    
    def __init__(self, df, data_key='data_words'):
        self.df = df
        self.df.reset_index(drop=True, inplace=True)
        self.data_key = data_key
    
    def create_topics(self, iterate_columns=None, limit=14):
        timer = time.time()
        topic_dict_list = []
        cont_dict_list = []
        for idx, (df, key_dict) in enumerate(self.df._topic.iterate(iterate_columns, return_keys=True)):
            print(f'{idx} - {datetime.timedelta(seconds=(time.time() - timer))}')
            dictionary = get_dictionary(df[self.data_key])
            corpus = get_corpus(dictionary, df[self.data_key])
            
            lda_model = get_optimised_lda_model(dictionary, corpus, df[self.data_key], limit=limit)
            ticket_counter, cont_dicts = self.get_dominant_topic(df, corpus, lda_model, offset=len(topic_dict_list))
            cont_dict_list += cont_dicts
            
            for topic_num in range(lda_model.num_topics):
                topic_dict_list.append(dict(topic_num=topic_num,
                                            topic_keywords=self.keywords_from_topic(lda_model, topic_num),
                                            topic_tickets=ticket_counter[topic_num],
                                            oldest_timestamp=df['sys_updated_on'].min(),
                                            newest_timestamp=df['sys_updated_on'].max(),
                                            **key_dict))
                
        return pd.DataFrame(topic_dict_list), pd.DataFrame(cont_dict_list)
            
    def get_dominant_topic(self, df, corpus, lda_model, offset=0):
        df_dict = df.to_dict(orient='records')
        ticket_counter = Counter()
        topic_contribution = []
        cont_dict_list = []
        for idx, row in enumerate(lda_model[corpus]):
            topic_num, contribution = max(row, key=lambda x: x[1])
            ticket_counter.update([topic_num])
            self.df.loc[df.iloc[idx].name, 'topic_num'] = topic_num
            self.df.loc[df.iloc[idx].name, 'unique_topic_num'] = offset + topic_num
            self.df.loc[df.iloc[idx].name, 'contribution'] = contribution
            self.df.loc[df.iloc[idx].name, 'topic_keywords'] = self.keywords_from_topic(lda_model, topic_num)
            for num, cont in row:
                cont_dict_list.append({**df_dict[idx], 'topic_num': num, 'unique_topic_num': offset + num, 'contribution': cont})
        
        return ticket_counter, cont_dict_list
            
            
    def keywords_from_topic(self, lda_model, topic_num):
        return ', '.join([w[0] for w in lda_model.show_topic(topic_num)])

In [ ]:
# cols = topic_out_df._topic.hierarchy
cols = ['u_category', 'contact_type']
topic_out_df, cont_out_df = topic_constructor.create_topics(cols, limit=14)

In [ ]:
_cols = cols + ['topic_num']

keyword_list = []
for idx, row in topic_out_df.iterrows():
    data_words = df._topic.dataframe_from_keys(_cols, [row[col] for col in _cols])['data_words']
    word_counter = Counter([word for r in data_words for word in r])
    
    for keyword in row['topic_keywords'].split(', '):
        keyword_list.append([keyword, idx, row['topic_num'],  word_counter[keyword]])
        
keyword_df = pd.DataFrame(keyword_list, columns=['keywords', 'unique_topic', 'topic_num', 'topic_keyword_count'])

# Remove keywords if < 2 characters and remove all keywords if < 2 keywords present
topic_out_df['topic_keywords'] = topic_out_df['topic_keywords'].apply(lambda x: [w for w in x.split(', ') if len(w) > 2]).apply(lambda x: '' if len(x) < 2 else ', '.join(x))

# topic_out_df.to_csv('topics.csv')
# keyword_df.to_csv('keywords.csv', index=False)

In [ ]:
topic_out_df